# Importações e Instalações de Biblioteca



In [1]:
!pip install 'aif360[Reductions,inFairness]'
!pip install xlrd


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [2]:
from BibliotecasEFuncoes import * 
from PredicaoParaEducation import * 

2024-05-21 14:25:35.715142: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 14:25:39.050873: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tuao/anaconda3/envs/aif360/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


# Lendo e processando a base de dados



Descrição Dos Dados no link: https://github.com/shrikant-temburwar/Loan-Prediction-Dataset

In [3]:

base = 'GitHubLoanContest'

GitHubLoan = pd.read_csv('Datasets/GitHubLoan-Prediction-Dataset.csv')
GitHubLoan.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
GitHubLoan.rename(columns={'Loan_Status' : 'class'}, inplace=True)
#separando rótulo (variável dependente y)
df_y = GitHubLoan['class']

## Codificando categorias

In [5]:
categorical_features = [i for i in GitHubLoan.columns if GitHubLoan.dtypes[i] in ['object', 'category']]
print(categorical_features)

['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'class']


In [6]:
categorical_names = {}
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  le.fit(GitHubLoan[feature])

  GitHubLoan[feature] = le.transform(GitHubLoan[feature])

  categorical_names[feature] = le.classes_
  encoders[feature] = le

In [7]:
GitHubLoan = GitHubLoan.dropna()
GitHubLoan = GitHubLoan.astype(int)

# todos devem estar codificados em int64
ghl = GitHubLoan
ghl.dtypes

Loan_ID              int64
Gender               int64
Married              int64
Dependents           int64
Education            int64
Self_Employed        int64
ApplicantIncome      int64
CoapplicantIncome    int64
LoanAmount           int64
Loan_Amount_Term     int64
Credit_History       int64
Property_Area        int64
class                int64
dtype: object

In [8]:
# Definições sobre os dados para avaliação de justiça
ghl.at[1, 'class'] = 0
label_name ='class'
protected_attribute_names = ['Gender', 'Education']
favorable_classes = np.where(ghl['class'] == 1)[0]

In [9]:
# Definindo as populações privilegiadas e não-privilegiadas
# Gênero
## Recuperando o indice
privileged_sex   = np.where(categorical_names['Gender'] == 'Male')[0]
unprivileged_sex = np.where(categorical_names['Gender'] == 'Female')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups   = [{'Gender' : privileged_sex}]
unprivileged_groups = [{'Gender' : unprivileged_sex}]

#Escolaridade
## Recuperando o indice
privileged_faixa_etaria   = np.where(categorical_names['Education'] == 'Graduate')[0]
unprivileged_faixa_etaria = np.where(categorical_names['Education'] == 'Not Graduate')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups_fe   = [{'Education' : privileged_faixa_etaria}]
unprivileged_groups_fe = [{'Education' : unprivileged_faixa_etaria}]


# Dados privilegiados
privileged_classes = [privileged_sex, privileged_faixa_etaria]

# Preparando experimentos

In [10]:
list_gbt_params, list_rf_params = preparando_experimentos()
print(list_gbt_params)

Total de 5 conjuntos de parâmetros.
Total de 2 conjuntos de parâmetros.
[{'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.2, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.3, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.4, 'max_depth': 2, 'random_state': 2024}, {'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.5, 'max_depth': 2, 'random_state': 2024}]


# Toda a base

## GBT

In [19]:
exp1_all_gbt = justica_exps_education(base, 'toda-a-base', ghl, list_gbt_params, 'gbt', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes, privileged_groups, unprivileged_groups, privileged_groups_fe, unprivileged_groups_fe)

<class 'dict'>
{'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}
0.82 de acuracia com desvio padrão de 0.07
0.75 de f1 com desvio padrão de 0.12
0.82 de precisão com desvio padrão de 0.06
0.77 de score no teste
STATISTICAL PARITY DIFFERENCE
Atributo sexo: 0.06947524020694751
Atributo educacao: 0.1315294117647059

DISPARATE IMPACT
Atributo sexo: 1.0955284552845528
Atributo educacao: 1.1944347826086956
 
AVERAGE ODDS DIFFERENCE
Atributo sexo: 0.04212454212454214
Atributo educacao: 0.0854655365004271
 
EQUAL OPPORTUNITY DIFFERENCE
Atributo sexo: 0.07142857142857151
Atributo educacao: 0.015375517445298637
{'index': 0, 'base': 'GitHubLoanContest', 'attr_set': 'toda-a-base', 'modelo': 'gbt', 'param_set': "{'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}", '10fold-acuracia': 0.8216216216216218, '10fold-f1': 0.74912894225134, '10fold-precisao': 0.8189895187774777, 'test-score': 0.

In [20]:
csv_result = pd.DataFrame(exp1_all_gbt, index=[4])
csv_result.to_csv('Resultados/04-GitHubLoanContest_GBT.csv')
print(csv_result)

   index               base     attr_set modelo                                          param_set  10fold-acuracia  10fold-f1  10fold-precisao  test-score  spd_sexo  spd_educacao   di_sexo  di_educacao  aod_sexo  aod_educacao  eod_sexo  eod_educacao
4      4  GitHubLoanContest  toda-a-base    gbt  {'n_estimators': 100, 'min_samples_split': 50,...          0.77027    0.71153         0.812959    0.792453 -0.016054      0.036434  0.978281     1.052049 -0.053184      0.021312 -0.052167     -0.008373


## RF


In [13]:
exp2_all_rf = justica_exps(base, 'toda-a-base', ghl, list_rf_params, 'rf', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes, unprivileged_groups, privileged_groups, unprivileged_groups_fe, privileged_groups_fe)

<class 'dict'>
{'n_estimators': 100, 'min_samples_split': 50, 'criterion': 'gini', 'max_depth': 2, 'random_state': 2024}
0.83 de acuracia com desvio padrão de 0.05
0.74 de f1 com desvio padrão de 0.10
0.82 de precisão com desvio padrão de 0.05
0.78 de score no teste
STATISTICAL PARITY DIFFERENCE
Atributo sexo: -0.030303030303030276
Atributo faixa_etaria: -0.07688113413304254

DISPARATE IMPACT
Atributo sexo: 0.9649122807017544
Atributo faixa_etaria: 0.9108723135271808
 
AVERAGE ODDS DIFFERENCE
Atributo sexo: -0.029040404040404033
Atributo faixa_etaria: -0.05258985200845667
 
EQUAL OPPORTUNITY DIFFERENCE
Atributo sexo: 0.022727272727272707
Atributo faixa_etaria: 0.022727272727272707
{'index': 0, 'base': 'GitHubLoanContest', 'attr_set': 'toda-a-base', 'modelo': 'rf', 'param_set': "{'n_estimators': 100, 'min_samples_split': 50, 'criterion': 'gini', 'max_depth': 2, 'random_state': 2024}", '10fold-acuracia': 0.827027027027027, '10fold-f1': 0.736773986216831, '10fold-precisao': 0.816219984935

In [18]:
csv_result2 = pd.DataFrame(exp2_all_rf, index=[9])
csv_result2.to_csv('Resultados/09-GitHubLoanContest_RF.csv')
print(csv_result2)

   index               base     attr_set modelo                                          param_set  10fold-acuracia  10fold-f1  10fold-precisao  test-score  spd_sexo  spd_faixa_etaria   di_sexo  di_faixa_etaria  aod_sexo  aod_faixa_etaria  eod_sexo  eod_faixa_etaria
9      1  GitHubLoanContest  toda-a-base     rf  {'n_estimators': 100, 'min_samples_split': 50,...         0.805405   0.709718         0.791901    0.836478  0.007939         -0.053396  1.009541         0.936026  0.073911         -0.006224  0.021505         -0.038764
